In [1]:
import numpy
import pandas
import joblib

# Instructions

- Read **the train data** from the CSV file and properly set the index
- Use `joblib` to load the trained model and print out the model parameters
- Put the model parameters in a DataFrame named `cluster_centers` with proper indices

In [2]:
data_train = pandas.read_csv('./data/features.train.csv').set_index('id')
data_train

feature_1  feature_2
id                       
253     2.1592     0.0103
667    -5.1157     0.8180
85      1.8528    -0.0038
969    -6.3364    -1.9423
75     -5.8828    -2.6060
..         ...        ...
835     2.1841     0.0434
192    -8.5461    -8.3446
629    -7.6355    -6.3077
559     2.0941     1.1079
684    -8.9178    -4.2373

[750 rows x 2 columns]

In [3]:
model = joblib.load('model/model.joblib')
model

KMeans(init='random', n_clusters=3, random_state=0)

In [4]:
print(f"model.cluster_centers_:\n {model.cluster_centers_}")

model.cluster_centers_:
 [[-4.86215514  0.10353131]
 [ 1.7568625   0.42275588]
 [-8.80245758 -5.38218182]]


In [5]:
cluster_centers = pandas.DataFrame(
    model.cluster_centers_,
    columns = ['feature_1', 'feature_2'],
)
cluster_centers.index.name = 'cluster'
cluster_centers

feature_1  feature_2
cluster                      
0        -4.862155   0.103531
1         1.756862   0.422756
2        -8.802458  -5.382182

# Instructions
- Use `.predict` to calculate clusters for all training samples
- Properly put the clusters into `pandas.DataFrame`, with a column named `cluster`

In [6]:
clusters_from_sklearn = model.predict(data_train[['feature_1', 'feature_2']])
clusters_from_sklearn


array([1, 0, 1, 0, 0, 2, 2, 2, 2, 1, 0, 2, 1, 1, 1, 2, 2, 1, 0, 2, 1, 2,
       1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 2, 2, 1, 0, 1, 2, 1, 0, 1, 2, 0, 0,
       2, 1, 0, 2, 1, 2, 2, 2, 2, 1, 0, 2, 1, 0, 2, 1, 2, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 1, 0, 0, 2, 0, 2, 1, 0, 1, 0,
       0, 0, 0, 2, 0, 0, 2, 2, 1, 2, 2, 1, 1, 1, 0, 2, 2, 2, 1, 2, 0, 1,
       2, 2, 1, 2, 2, 1, 1, 2, 0, 1, 2, 1, 2, 1, 0, 0, 1, 0, 2, 1, 2, 0,
       0, 2, 0, 0, 2, 1, 0, 0, 2, 1, 2, 1, 0, 1, 1, 2, 0, 1, 0, 2, 0, 1,
       0, 0, 0, 2, 1, 1, 2, 0, 2, 2, 2, 0, 0, 1, 1, 2, 0, 1, 2, 1, 2, 2,
       0, 1, 0, 2, 2, 2, 0, 0, 0, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2,
       1, 1, 0, 2, 0, 1, 2, 2, 2, 2, 0, 0, 2, 0, 1, 0, 1, 1, 2, 1, 2, 0,
       0, 2, 0, 1, 1, 0, 2, 1, 2, 0, 2, 1, 1, 2, 2, 1, 1, 0, 0, 0, 0, 1,
       2, 2, 2, 2, 2, 2, 0, 1, 0, 0, 1, 1, 2, 0, 2, 1, 2, 1, 2, 1, 1, 0,
       1, 2, 2, 1, 1, 0, 2, 0, 1, 1, 2, 0, 0, 1, 1, 2, 2, 0, 0, 2, 1, 1,
       0, 0, 0, 0, 0, 1, 2, 0, 2, 2, 2, 2, 1, 0, 1,

In [7]:
clusters_from_sklearn = pandas.DataFrame(
    data = clusters_from_sklearn,
    index = data_train.index,
    columns = ['cluster']
)
clusters_from_sklearn

cluster
id          
253        1
667        0
85         1
969        0
75         0
..       ...
835        1
192        2
629        2
559        1
684        2

[750 rows x 1 columns]

# Instructions
- Write a function that calculate the distances from a given sample point to the cluster centers
  - Hint: use the DataFrame `cluster_centers`, `.apply`
- Use the function to calculate the distances from a sample point at [0., 0.]
  - Also, indicate which cluster the sample belongs to
- Now, **manually calculate** the clusters for all training samples
  - Also, properly put the data into `pandas.DataFrame`, with a column named `cluster`
- Verify that the clusters from `.predict` and manual calculations are equal


In [8]:
def calculate_distances_from_cluster_centers(sample):
    return (
        (sample - cluster_centers)
        .apply(numpy.square)
        .sum(axis='columns')
        .apply(numpy.sqrt)
    )    

In [9]:
calculate_distances_from_cluster_centers([0, 0])


cluster
0     4.863257
1     1.807011
2    10.317516
dtype: float64

In [10]:
distances_from_calculations = (
    data_train[['feature_1', 'feature_2']]
    .apply(calculate_distances_from_cluster_centers, axis='columns')
)
distances_from_calculations

cluster         0          1          2
id                                     
253      7.021974   0.576190  12.216251
667      0.758123   6.883918   7.213490
85       6.715813   0.437212  11.935724
969      2.521671   8.431749   4.232520
75       2.895389   8.218139   4.028844
..            ...        ...        ...
835      7.046512   0.571352  12.253219
192      9.216419  13.528398   2.973490
629      6.985365  11.554891   1.489421
559      7.028388   0.763644  12.682907
684      5.940629  11.647512   1.150677

[750 rows x 3 columns]

In [11]:
clusters_from_calculation = distances_from_calculations.idxmin(axis='columns')

In [12]:
clusters_from_calculation = pandas.DataFrame(
    clusters_from_calculation,
    columns = ['cluster']
)
clusters_from_calculation

cluster
id          
253        1
667        0
85         1
969        0
75         0
..       ...
835        1
192        2
629        2
559        1
684        2

[750 rows x 1 columns]

In [13]:
(clusters_from_sklearn == clusters_from_calculation).all()

cluster    True
dtype: bool